In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update > /dev/null
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [4]:
# Start a SparkSession
import findspark
findspark.init()

In [5]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar > /dev/null

--2022-05-04 16:03:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.91MB/s    in 0.2s    

2022-05-04 16:03:33 (4.91 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Challenge_16").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://bucketchallenge16.s3.us-west-1.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)

In [8]:
df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1LXDUB11LTYEZ|          5|            0|          0|   N|                Y|
|R3GOJDK6SSS4KK|          5|            0|          0|   N|                Y|
| RL7840L7ZSZIW|          5|            3|          3|   N|                N|
| RADV8PV12RTEE|          5|            0|          0|   N|                Y|
|R3JXYWAEP2YCW5|          4|            1|          1|   N|                Y|
|R2DADRV9FENU73|          5|            0|          0|   N|                Y|
| R5X7B5IMNM0U6|          5|            0|          1|   N|                Y|
|R3URXFRR0GUT5D|          2|            1|          1|   N|                Y|
| R9MXL31N81TFV|          5|            0|          0|   N|                Y|
| RMQDJL8F0BD4B|          5|            0|          0|   N|     

In [9]:
df_filtered = df[df.total_votes > 20]
df_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2CSA8HYEJPMF5|          5|           29|         29|   N|                Y|
|R2OETDNMAH8ED3|          5|           48|         51|   N|                Y|
|R1C7LSXM2CG029|          1|           32|         39|   N|                Y|
|R2BK2XNBQ0VEFK|          1|            2|         74|   N|                N|
|R3BPULXDSGMIFL|          5|          374|        398|   N|                Y|
|R1VPUNU79C377J|          5|           25|         25|   N|                Y|
| R4N4NDFZIO2SV|          5|           42|         59|   N|                Y|
|R10LM1QET8CQQQ|          4|           24|         27|   N|                Y|
| RP08F9RIGHEV7|          5|           36|         36|   N|                Y|
|R36BV2R9T7U10L|          4|           25|         27|   N|     

In [10]:
div_df = df_filtered[(df_filtered.helpful_votes/df_filtered.total_votes) >= 0.5]
print(div_df.count())
div_df.show()

57920
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2CSA8HYEJPMF5|          5|           29|         29|   N|                Y|
|R2OETDNMAH8ED3|          5|           48|         51|   N|                Y|
|R1C7LSXM2CG029|          1|           32|         39|   N|                Y|
|R3BPULXDSGMIFL|          5|          374|        398|   N|                Y|
|R1VPUNU79C377J|          5|           25|         25|   N|                Y|
| R4N4NDFZIO2SV|          5|           42|         59|   N|                Y|
|R10LM1QET8CQQQ|          4|           24|         27|   N|                Y|
| RP08F9RIGHEV7|          5|           36|         36|   N|                Y|
|R36BV2R9T7U10L|          4|           25|         27|   N|                N|
|R1B8UEBSD9EPJI|          5|           36|         40|   N

In [11]:
vine_program_df = div_df[div_df.verified_purchase == 'Y']
print(vine_program_df.count())
vine_program_df.show()

39645
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2CSA8HYEJPMF5|          5|           29|         29|   N|                Y|
|R2OETDNMAH8ED3|          5|           48|         51|   N|                Y|
|R1C7LSXM2CG029|          1|           32|         39|   N|                Y|
|R3BPULXDSGMIFL|          5|          374|        398|   N|                Y|
|R1VPUNU79C377J|          5|           25|         25|   N|                Y|
| R4N4NDFZIO2SV|          5|           42|         59|   N|                Y|
|R10LM1QET8CQQQ|          4|           24|         27|   N|                Y|
| RP08F9RIGHEV7|          5|           36|         36|   N|                Y|
|R1B8UEBSD9EPJI|          5|           36|         40|   N|                Y|
|R36C6M0T03EG63|          4|           59|         61|   N

In [12]:
not_vine_program_df = div_df[div_df.verified_purchase == 'N']
print(not_vine_program_df.count())
not_vine_program_df.show()

18275
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R36BV2R9T7U10L|          4|           25|         27|   N|                N|
|R31FJE3EZ4UTD8|          5|           62|         70|   N|                N|
|R10CLK3SA78KYR|          1|           24|         35|   N|                N|
|R32YMQ84KK0PX7|          2|           43|         51|   N|                N|
| RIWPG8D08WLY8|          5|           30|         33|   N|                N|
|R10C6BQTN0DTBC|          3|           34|         39|   N|                N|
|R1B0618SXG4TS2|          4|           31|         33|   N|                N|
|R2T1SS1EU4D4D6|          5|           70|         77|   N|                N|
| R7WV9UR7XWQAM|          5|           46|         52|   N|                N|
|R2NDD6RXINJEEZ|          1|           30|         38|   N

In [13]:
# paid total number of reviews
total_paid_reviews = vine_program_df.count()
total_paid_reviews

39645

In [14]:
# paid 5-star reviews
paid_five_star_reviews = vine_program_df.filter(vine_program_df.star_rating == 5).count()
paid_five_star_reviews


21361

In [15]:
# paid 5-star reviews percentage
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

53.88069113381258

In [16]:
# unpaid total number of reviews
total_unpaid_reviews = not_vine_program_df.count()
total_unpaid_reviews

18275

In [17]:
# unpaid 5-star reviews
unpaid_five_star_reviews = not_vine_program_df.filter(not_vine_program_df.star_rating == 5).count()
unpaid_five_star_reviews

9411

In [18]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

51.496580027359784